## DataFrame


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
Ollama_URL = os.getenv("Ollama_URL")

In [ ]:

import pandas as pd

file_path = "Vintage_Lane.csv"
df = pd.read_csv(file_path)

# Drop unnecessary columns
drop_cols = [
    "Image link (comma seperated)",
    "Item Subcategories",
    "Shopping Subcategories",
    "Unnamed: 14",
    "Shoiping categories",
    "Variation Price (Incl. VAT EGP)"
]
df = df.drop(columns=drop_cols, errors='ignore')

# Clear all Ollama() output columns
for col in df.columns:
    if "- OLLAMA()" in col:
        df[col] = ""

df.to_csv("items_cleaned.csv", index=False)

In [3]:
df

,Item (EN),Description (EN),Category/Department (EN),item Categories,Variant Name,SKW - input(),SKW - OLLAMA(),DSW - input(),DSW - OLLAMA(),Ai Attributes - input(),Ai Attributes - OLLAMA()
0,1920S ANTIQUE MONETE ROMAN IMPERIAL COIN 18K B...,NaN,All Collection,ring,NaN,Item name: 1920S ANTIQUE MONETE ROMAN IMPERIAL...,,Item name: 1920S ANTIQUE MONETE ROMAN IMPERIAL...,,Item name: 1920S ANTIQUE MONETE ROMAN IMPERIAL...,
1,1938 Royal Wedding Pin – King Farouk & Queen F...,Not available for international shipping,Gentlemen’S Collection,broche,NaN,Item name: 1938 Royal Wedding Pin – King Farou...,,Item name: 1938 Royal Wedding Pin – King Farou...,,Item name: 1938 Royal Wedding Pin – King Farou...,
2,1950S-1970S Era W Germany Cherry Red Faceted B...,NaN,All Collection,necklace,NaN,Item name: 1950S-1970S Era W Germany Cherry Re...,,Item name: 1950S-1970S Era W Germany Cherry Re...,,Item name: 1950S-1970S Era W Germany Cherry Re...,
3,1960S Vintage Black & Grey Daisy Flower Brooch...,NaN,All Collection,broche,NaN,Item name: 1960S Vintage Black & Grey Daisy Fl...,,Item name: 1960S Vintage Black & Grey Daisy Fl...,,Item name: 1960S Vintage Black & Grey Daisy Fl...,
4,1970S Gold-Plated Fish Stud Earrings – Made In...,NaN,All Collection,earring,NaN,Item name: 1970S Gold-Plated Fish Stud Earring...,,Item name: 1970S Gold-Plated Fish Stud Earring...,,Item name: 1970S Gold-Plated Fish Stud Earring...,
5,1970S Heraldic Gold Tone Rhinestone Fleur-De-L...,NaN,STEAL THE LOOK,broche,NaN,Item name: 1970S Heraldic Gold Tone Rhinestone...,,Item name: 1970S Heraldic Gold Tone Rhinestone...,,Item name: 1970S Heraldic Gold Tone Rhinestone...,
6,1970S Vintage British Designer – Sleeping Cat ...,Feminine • Elegance • Sophistication\nDiscover...,All Collection,broche,NaN,Item name: 1970S Vintage British Designer – Sl...,,Item name: 1970S Vintage British Designer – Sl...,,Item name: 1970S Vintage British Designer – Sl...,
7,1980S Vintage Door Knocker Clip-On Earrings – ...,Wear history with style—VINTAGE LANE crafts ex...,All Collection,earring,NaN,Item name: 1980S Vintage Door Knocker Clip-On ...,,Item name: 1980S Vintage Door Knocker Clip-On ...,,Item name: 1980S Vintage Door Knocker Clip-On ...,
8,1980S Vintage Napier Gold Tone Accents And Fau...,NaN,NECKLACES,necklace,NaN,Item name: 1980S Vintage Napier Gold Tone Acce...,,Item name: 1980S Vintage Napier Gold Tone Acce...,,Item name: 1980S Vintage Napier Gold Tone Acce...,
9,"90S Power Glam, Bold Gold Elegance",Feminine • Elegance • Sophistication\nDiscover...,STEAL THE LOOK,earring,NaN,"Item name: 90S Power Glam, Bold Gold Elegance\...",,"Item name: 90S Power Glam, Bold Gold Elegance\...",,"Item name: 90S Power Glam, Bold Gold Elegance\...",


## Ollama

In [ ]:
import requests

def ollama_local(prompt):
    url = Ollama_URL
    payload = {
        "model": "llama3.1:8b",
        "prompt": prompt,
        "max_tokens": 200,
        "stream": False
    }
    headers = {"Content-Type": "application/json"}

    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    data = response.json()
    
    # Extract only the generated text
    return data["choices"][0]["text"].strip()


In [5]:
# Example usage
output = ollama_local("Hello from Python using local Ollama!")
print(output)

Hello from the other side (of the server, that is)!. How can I assist you with Python today?


## Open AI

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)  

def chatgpt(prompt, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200
    )
    return response.choices[0].message.content.strip()

# Test
output = chatgpt("Hello from Python using GPT-3.5 turbo!")
print(output)


Hello! How can I assist you today?


## Transformation


In [6]:
pd.set_option('display.max_colwidth', None)

### SKW

In [7]:
print(df['SKW -  input()'].loc[0])

Item name: 1920S ANTIQUE MONETE ROMAN IMPERIAL COIN 18K BOLD RING
Vendor Category: All Collection


In [8]:
# Get the first row
row = df.iloc[0]

# SKW
if pd.notna(row["SKW -  input()"]):
    skw_prompt = (
        """Generate 5 shopping keyword phrases for this item. 
Rules:
- Do not use numbering or extra text, only list the phrases.
- The first phrase must be only the main product (example: "Ring").
- All other phrases must end with the main product word.
- Each phrase must be maximum 3 words only. Never exceed 3 words.
- Format = modifier + modifier + main product.
- Modifiers = tangible features or proper nouns.
- Remove sentiments, numbers, dates, symbols.
Item data: """
        + row["SKW -  input()"]
    )
    df.at[0, "SKW - OLLAMA()"] = ollama_local(skw_prompt)

print(df.loc[0, "SKW - OLLAMA()"])


Gold Coin Ring
Greek Imperial Ring
Vintage Roman Ring
Monet Antique Ring
Old Empire Ring


In [ ]:
# Get the first row
row = df.iloc[0]

# SKW
if pd.notna(row["SKW -  input()"]):
    skw_prompt = (
        """Generate 5 shopping keyword phrases for this item. 
Rules:
- Do not use numbering or extra text, only list the phrases.
- The first phrase must be only the product category (example: "Ring", "Necklace", etc.).
- All other phrases must end with the product category word.
- Each phrase must be maximum 3 words only. Never exceed 3 words.
- Format = modifier + modifier + product category.
- Modifiers = tangible features or proper nouns.
- Remove sentiments, numbers, dates, symbols.
Item data: """
        + row["SKW -  input()"]
    )
    df.at[0, "SKW - OLLAMA()"] = chatgpt(skw_prompt)

print(df.loc[0, "SKW - OLLAMA()"])


Coin Bold Ring
Antique Roman Ring
Imperial Coin Ring
Monete Gold Ring
1920s Antique Ring


### DSW

In [9]:
df['DSW -  input()'].loc[0]

'Item name: 1920S ANTIQUE MONETE ROMAN IMPERIAL COIN 18K BOLD RING\nVendor Category: All Collection'

In [10]:
# DSW
if pd.notna(row["DSW -  input()"]):
    dsw_prompt = (
    """Generate 5–10 shopping keyword phrases for this item.  
    Rules:  
    - Do not use numbering, bullets, or extra comments, only list the phrases.  
    - Each phrase must end with the main product word.  
    - Format (≤3 words) = modifier + modifier + main product.  
    - Modifiers = tangible features, functional attributes, or proper nouns.  
    - Include exactly one phrase with only the main product (example: "Ring").  
    - Remove sentiments, opinions, numbers, dates, symbols, and abbreviations.  
    Item data: """
        + row["DSW -  input()"]
    )
    df.at[0, "DSW - OLLAMA()"] = ollama_local(dsw_prompt)

print(df.loc[0, "DSW - OLLAMA()"])

antique gold ring
imperial 18k ring
roman coin ring
bold 18k ring 
 antique roman ring 
golden imperial ring
genuine antique ring
vintage roman imperial ring
roman empire 18k ring
monete 18k ring


### AI Attributes

In [11]:
df['Ai Attributes -  input()'].loc[0]

'Item name: 1920S ANTIQUE MONETE ROMAN IMPERIAL COIN 18K BOLD RING\nVendor Category: All Collection\nItem Category: ring'

In [36]:
# AI Attributes
if pd.notna(row["Ai Attributes -  input()"]):
    ai_prompt = (
        f"Item data: {row['Ai Attributes -  input()']}"
        + """
- Identify all attributes of this item. Only fill attributes that can be clearly inferred from the data; leave unknowns empty. Do not include extra statements, explanations, or brackets. Use short, precise values in English.  

- Gender must be chosen strictly from this list only:  
  ["Women", "Men", "Unisex women, Unisex men", "Girls", "Boys", "Unisex girls, unisex boys"].  
  Do not invent or shorten values (e.g., "Unisex men" is invalid).  

- Generic Name should use the item category if possible.  
- Color should be inferred from the item name or category if present.  
- Product Name should be concise and represent the product, not copied verbatim.  

Format exactly as:

Gender: 
Age: 
Brand: 
Generic Name: 
Product Name: 
Size: 
Measurements: 
Features: 
Types of Fashion Styles: 
Gem Stones: 
Birth Stones: 
Material: 
Color: 
Pattern: 
Occasion: 
Activity: 
Season: 
Country of origin: 
"""
    )
    df.at[0, "Ai Attributes - OLLAMA()"] = chatgpt(ai_prompt)

print(df.loc[0, "Ai Attributes - OLLAMA()"])

Gender: Unisex women, Unisex men
Age: 
Brand: 
Generic Name: Roman Imperial Coin Ring
Product Name: Monete Roman Coin 18K Ring
Size: 
Measurements: 
Features: 
Types of Fashion Styles: Vintage, Antique
Gem Stones: 
Birth Stones: 
Material: 18K Gold
Color: Gold
Pattern: 
Occasion: 
Activity: 
Season: 
Country of origin:


In [35]:
print(f"Item data: {row['Ai Attributes -  input()']}")

Item data: Item name: 1920S ANTIQUE MONETE ROMAN IMPERIAL COIN 18K BOLD RING
Vendor Category: All Collection
Item Category: ring


In [ ]:
import requests

url = Ollama_URL
API_MODEL = "mistral"

def test_mistral(prompt: str):
    payload = {
        "model": API_MODEL,
        "prompt": prompt,
        "stream": False
    }
    response = requests.post(url, json=payload)
    response.raise_for_status()
    return response.json().get("response", "").strip()


In [1]:
# print(test_mistral("Hello from Python to Mistral!"))